In [1]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [17]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [25]:
includet("utils.jl")
includet("VAR_OLS.jl")

In [19]:
df = DataFrame(XLSX.readtable("../../data/usa_world_data.xlsx", "Arkusz1", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

146×15 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-03-01 to 2024-04-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1988-03-01 │   0.0857 │   677.401 │       262.6 │  9.04333 │  18.59 │ 14.433 ⋯
│ 1988-06-01 │   0.0882 │   681.206 │       269.1 │  9.30667 │  21.67 │ 14.608 ⋯
│ 1988-09-01 │   0.0887 │   694.387 │       277.4 │     9.55 │  22.73 │ 14.811 ⋯
│ 1988-12-01 │   0.0914 │   697.568 │       285.4 │  9.81333 │  23.75 │ 15.088 ⋯
│ 1989-03-01 │    0.093 │   706.377 │       302.3 │  10.0867 │  24.96 │ 15.686 ⋯
│ 1989-06-01 │    0.081 │    780.73 │       331.9 │  10.4233 │  25.22 │ 17.013 ⋯
│ 1989-09-01 │   0.0831 │   785.343 │       347.4 │  10.7967 │  23.69 │ 17.640 ⋯
│ 1989-12-01 │   0.0793 │   821.683 │      339.97 │    11.14 │  22.87 │ 17.048 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [54]:
freq = 4
returns = percentchange(data_source[:ACWI_Net, :USA_Net, :Price_10Y, :World_ex_USA, :EM_Net], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
real_t_bill = log1p.(data_source[:T_Bill])./freq .- cpi
real_10Y = log1p.(data_source[:Yeld_10Y])./freq .- cpi
excess_returns = returns .- log1p.(data_source[:T_Bill]./freq)
dy = log.(data_source[:Dividend_Yeld])
term = log1p.(data_source[:Yeld_10Y] ./ freq)  .- log1p.(data_source[:T_Bill] ./ freq)
#data = merge(real_returns, real_t_bill, term, cpi, dy)
data = real_returns

start = Date(1989, 01,01)
end_d = Date(2019, 12, 31)
s_data = to(from(data, start),end_d)

display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))
display("Excess_returns")
display(returns_summarystats(to(from(excess_returns, start),end_d), freq))

display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
ACWI_Net,0.1572,1.1429,0.0669,0.2358,-1.6892,-1.3615,-0.0588,0.1114,7.5188,0.4256,0.1125
USA_Net,0.1499,0.9025,0.0949,0.283,-1.4498,-1.2991,-0.0239,0.1164,5.2866,0.6329,0.1145
Price_10Y,0.0798,0.5691,0.0629,0.1747,-0.2868,0.3575,-0.0545,-0.02,0.0554,0.7875,0.0459
World_ex_USA,0.1728,1.2168,0.0466,0.2697,-1.7796,-0.9837,-0.1468,0.1181,5.7161,0.2697,0.08
EM_Net,0.2638,1.7838,0.089,0.4038,-2.3616,-0.9186,-0.1935,0.0625,4.0193,0.3375,0.1546


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
ACWI_Net_CPI,0.1538,1.1035,0.0425,0.2046,-1.5876,-1.1513,-0.0796,0.0975,6.1829,0.2764,0.0671
USA_Net_CPI,0.1477,0.8631,0.0705,0.2568,-1.3483,-1.1233,-0.0474,0.0946,4.108,0.4772,0.0922
Price_10Y_CPI,0.0842,0.6707,0.0385,0.1522,-0.3051,0.4734,-0.0882,-0.031,0.6284,0.4567,0.0263
World_ex_USA_CPI,0.1689,1.1774,0.0222,0.2332,-1.678,-0.7977,-0.1597,0.1098,4.7191,0.1315,0.0535
EM_Net_CPI,0.2593,1.7444,0.0646,0.3797,-2.26,-0.8337,-0.208,0.0543,3.6406,0.2493,0.1253


...

"Excess_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
ACWI_Net_T_Bill,0.1573,1.1411,0.0385,0.2029,-1.6895,-1.2483,-0.0963,0.1134,7.0986,0.2447,0.0729
USA_Net_T_Bill,0.1491,0.9007,0.0665,0.2387,-1.4501,-1.2424,-0.0497,0.1084,5.0526,0.4457,0.0939
Price_10Y_T_Bill,0.079,0.5688,0.0344,0.1376,-0.2919,0.4425,-0.0856,-0.0456,0.2657,0.4359,0.0241
World_ex_USA_T_Bill,0.1732,1.215,0.0182,0.232,-1.7799,-0.8814,-0.1853,0.1223,5.3542,0.1051,0.0529
EM_Net_T_Bill,0.2633,1.782,0.0606,0.3693,-2.3619,-0.9216,-0.2122,0.0598,3.9818,0.2302,0.1164


nothing

"Correlations"

5×5 Matrix{Float64}:
  1.0        0.910927  -0.451822   0.96631    0.74476
  0.910927   1.0       -0.38457    0.785911   0.62635
 -0.451822  -0.38457    1.0       -0.427688  -0.485259
  0.96631    0.785911  -0.427688   1.0        0.713031
  0.74476    0.62635   -0.485259   0.713031   1.0

In [55]:
display(returns_summarystats(to(from(merge(real_t_bill, real_10Y, dy, term), start),end_d), freq))

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
T_Bill_CPI,0.0156,0.1019,0.0035,0.0249,-0.0832,-0.0999,-0.0158,0.1559,0.5091,0.2268,0.0068
Yeld_10Y_CPI,0.0147,0.1238,0.0203,0.0388,-0.0555,-0.0106,0.0048,0.0652,0.9311,1.3771,0.0238
Dividend_Yeld,0.5563,-12.9938,-15.6017,-15.0879,-17.9466,0.164,-16.1656,0.9531,-0.3427,-28.045,-15.7176
Yeld_10Y_T_Bill,0.0055,0.0378,0.0172,0.0261,-0.0064,-0.0719,0.0078,0.889,-1.0183,3.0929,0.0175


nothing

In [56]:
start = Date(1989, 01, 01)
end_d = Date(2024, 01, 31)
assets_names = [:USA_Net_T_Bill, :World_ex_USA_T_Bill, :EM_Net_T_Bill]
data = to(from(merge(excess_returns[assets_names], real_t_bill, dy, term), start),end_d)
assets_names = colnames(data)
T, n_assets = size(data)
model = VARModel(data)
model_summary(model)

,const,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill,std
USA_Net_T_Bill,0.1697,0.1248,0.0204,-0.0203,-1.6295,0.0354,-3.9738,0.0724
World_ex_USA_T_Bill,-0.0233,0.0263,0.018,0.0578,-0.9841,-0.0081,-1.233,0.0845
EM_Net_T_Bill,0.2301,-0.2145,0.2693,0.0243,-0.2681,0.0531,-1.6701,0.1275
T_Bill_CPI,0.0274,-0.0004,0.0095,-0.0027,0.2479,0.0063,-0.6251,0.0081
Dividend_Yeld,-0.2257,0.0248,-0.1052,0.0341,-0.9466,0.947,2.7692,0.0736
Yeld_10Y_T_Bill,0.0022,-0.0017,-0.0011,0.0014,-0.0069,0.0005,0.9021,0.0012


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
USA_Net_T_Bill,1.0,0.8,0.65,-0.13,-0.59,0.21
World_ex_USA_T_Bill,0.8,1.0,0.74,-0.26,-0.48,0.18
EM_Net_T_Bill,0.65,0.74,1.0,-0.26,-0.32,0.31
T_Bill_CPI,-0.13,-0.26,-0.26,1.0,0.08,-0.2
Dividend_Yeld,-0.59,-0.48,-0.32,0.08,1.0,-0.08
Yeld_10Y_T_Bill,0.21,0.18,0.31,-0.2,-0.08,1.0


In [57]:
scenarios = simulate(model, 100, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(periods), string.(assets_names[a]))
end  

,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
1,0.0708,0.0241,0.0124,0.0224,-16.9976,-0.0108
5,0.0808,0.0423,0.007,0.0068,-17.1272,-0.0026
15,0.0704,0.0391,0.0088,0.0009,-16.9333,0.0041
25,0.0684,0.037,0.0135,-0.0002,-16.784,0.0064


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
1,0.1363,0.1615,0.2336,0.0168,0.2646,0.0044
5,0.0684,0.0867,0.1177,0.0114,0.5645,0.0076
15,0.0402,0.0538,0.0709,0.0075,0.6907,0.0078
25,0.0312,0.0429,0.057,0.006,0.673,0.0072


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
1,-0.0,0.037,-0.0073,-0.0302,-0.0128,-0.0041
5,0.0302,0.0144,0.0103,0.0047,0.0199,0.0084
15,0.0241,0.0256,0.016,-0.0371,0.0141,0.0425
25,0.052,0.0234,0.0037,-0.0076,-0.0028,0.0255


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill
1,0.0241,0.0522,0.0616,-0.0194,-0.0878,-0.0611
5,0.0166,-0.0161,-0.0328,-0.0288,0.0507,-0.0714
15,-0.0439,-0.02,0.0179,-0.0215,0.0081,0.0516
25,0.0195,-0.0168,-0.0458,0.0265,-0.0162,-0.0074


,0.03,0.25,0.5,0.75,0.97
1,-0.1844,-0.0214,0.0704,0.1618,0.3275
5,-0.0464,0.0337,0.0803,0.1265,0.2119
15,-0.0049,0.0429,0.0703,0.0975,0.1469
25,0.0107,0.0473,0.0683,0.0893,0.1277


,0.03,0.25,0.5,0.75,0.97
1,-0.2761,-0.0854,0.0219,0.1325,0.3295
5,-0.1195,-0.0156,0.0425,0.1,0.2079
15,-0.0622,0.003,0.0393,0.0754,0.1407
25,-0.0435,0.0081,0.0364,0.0661,0.1178


,0.03,0.25,0.5,0.75,0.97
1,-0.4231,-0.1442,0.0111,0.1697,0.448
5,-0.2134,-0.0739,0.0082,0.0854,0.2297
15,-0.1234,-0.0385,0.0084,0.0561,0.1431
25,-0.0937,-0.0254,0.0133,0.0527,0.1195


,0.03,0.25,0.5,0.75,0.97
1,-0.0098,0.0112,0.0224,0.0338,0.054
5,-0.0145,-0.0008,0.0067,0.0146,0.0282
15,-0.0134,-0.0042,0.0011,0.0061,0.0149
25,-0.0114,-0.0042,-0.0002,0.0039,0.011


,0.03,0.25,0.5,0.75,0.97
1,-17.4914,-17.1788,-16.995,-16.8177,-16.4962
5,-18.1888,-17.5069,-17.1315,-16.7508,-16.0585
15,-18.2253,-17.3926,-16.9341,-16.4687,-15.6362
25,-18.0438,-17.2404,-16.7825,-16.3347,-15.5278


,0.03,0.25,0.5,0.75,0.97
1,-0.0189,-0.0138,-0.0108,-0.0078,-0.0026
5,-0.0169,-0.0078,-0.0026,0.0024,0.0118
15,-0.0105,-0.0012,0.0041,0.0093,0.019
25,-0.0071,0.0015,0.0064,0.0113,0.0199


In [58]:
start = Date(1989, 01, 01)
end_d = Date(2024, 01, 31)
assets_names = [:USA_Net_T_Bill, :Price_10Y_T_Bill, :World_ex_USA_T_Bill, :EM_Net_T_Bill]
data = to(from(merge(excess_returns[assets_names], real_t_bill, dy), start),end_d)
assets_names = colnames(data)
T, n_assets = size(data)
model = VARModel(data)
model_summary(model)

,const,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld,std
USA_Net_T_Bill,0.0895,0.1488,0.1377,0.0134,-0.0154,-1.3644,0.0193,0.073
Price_10Y_T_Bill,0.0639,0.077,-0.0663,-0.0155,-0.0312,0.6806,0.0147,0.0413
World_ex_USA_T_Bill,-0.0463,0.0307,0.0167,0.0169,0.0567,-0.8777,-0.0126,0.0846
EM_Net_T_Bill,0.1904,-0.1945,0.1415,0.2628,0.0347,-0.2343,0.045,0.1275
T_Bill_CPI,0.0184,-0.0026,-0.0287,0.0105,-0.007,0.3363,0.0046,0.0082
Dividend_Yeld,-0.1668,0.0032,-0.1374,-0.0985,0.0265,-1.0929,0.9588,0.0738


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
USA_Net_T_Bill,1.0,-0.38,0.8,0.65,-0.09,-0.59
Price_10Y_T_Bill,-0.38,1.0,-0.36,-0.43,0.25,0.2
World_ex_USA_T_Bill,0.8,-0.36,1.0,0.74,-0.25,-0.48
EM_Net_T_Bill,0.65,-0.43,0.74,1.0,-0.25,-0.32
T_Bill_CPI,-0.09,0.25,-0.25,-0.25,1.0,0.05
Dividend_Yeld,-0.59,0.2,-0.48,-0.32,0.05,1.0


In [59]:
scenarios = simulate(model, 100, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(periods), string.(assets_names[a]))
end  

,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
1,0.0264,0.0637,0.0139,0.0024,0.0087,-16.9118
5,0.0467,0.0198,0.0287,0.009,-0.0025,-16.7218
15,0.0539,0.0169,0.0282,0.0194,-0.0031,-16.4754
25,0.0564,0.0167,0.028,0.024,-0.003,-16.3855


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
1,0.1379,0.0715,0.16,0.2342,0.0176,0.2681
5,0.0712,0.0369,0.0874,0.1182,0.0101,0.5636
15,0.0404,0.0239,0.0543,0.0696,0.0065,0.6141
25,0.0308,0.0192,0.0429,0.0539,0.0052,0.5464


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
1,0.0355,-0.0362,0.0261,0.016,0.0037,-0.0045
5,-0.0185,0.014,-0.0035,0.0253,-0.0222,-0.0292
15,0.0301,-0.0234,0.0451,0.0054,-0.0034,0.0294
25,0.0116,-0.0203,0.0128,-0.0196,-0.0473,0.0066


,USA_Net_T_Bill,Price_10Y_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Dividend_Yeld
1,-0.0315,0.0135,-0.0133,-0.0293,0.0327,-0.0587
5,-0.063,-0.0064,0.001,-0.0082,0.1024,-0.0857
15,-0.0051,-0.0426,-0.0699,-0.0258,0.0908,-0.0159
25,0.0244,-0.0389,0.0717,0.0885,0.0153,-0.0131


,0.03,0.25,0.5,0.75,0.97
1,-0.2305,-0.0674,0.0245,0.1187,0.2883
5,-0.0863,-0.001,0.0468,0.0955,0.1803
15,-0.0207,0.0271,0.0531,0.0811,0.13
25,-0.0011,0.0359,0.0565,0.0771,0.1148


,0.03,0.25,0.5,0.75,0.97
1,-0.0741,0.0157,0.0632,0.1123,0.1955
5,-0.0498,-0.0047,0.0194,0.045,0.0888
15,-0.0282,0.0008,0.0169,0.0332,0.0612
25,-0.0193,0.0038,0.0167,0.0297,0.0528


,0.03,0.25,0.5,0.75,0.97
1,-0.2864,-0.0941,0.0116,0.1208,0.3178
5,-0.1365,-0.0292,0.028,0.0865,0.1945
15,-0.0729,-0.0086,0.0275,0.0653,0.1303
25,-0.0543,-0.0005,0.028,0.0567,0.1082


,0.03,0.25,0.5,0.75,0.97
1,-0.436,-0.1534,0.0014,0.1605,0.4486
5,-0.2088,-0.0716,0.0079,0.0876,0.2332
15,-0.1095,-0.0277,0.0198,0.0666,0.1482
25,-0.0784,-0.0126,0.0244,0.0604,0.1241


,0.03,0.25,0.5,0.75,0.97
1,-0.0244,-0.0034,0.0088,0.0203,0.0421
5,-0.0218,-0.0092,-0.0027,0.0044,0.0163
15,-0.0154,-0.0074,-0.0031,0.0012,0.009
25,-0.0128,-0.0065,-0.003,0.0005,0.0066


,0.03,0.25,0.5,0.75,0.97
1,-17.4138,-17.0934,-16.9113,-16.7282,-16.4089
5,-17.8027,-17.0993,-16.7193,-16.3426,-15.6466
15,-17.6208,-16.8923,-16.4803,-16.056,-15.3241
25,-17.399,-16.7602,-16.3864,-16.0077,-15.363


In [ ]:
start = Date(1989, 01, 01)
end_d = Date(2024, 01, 31)
assets_names = [:USA_Net_T_Bill, :World_ex_USA_T_Bill, :EM_Net_T_Bill]
data = to(from(merge(excess_returns[assets_names], real_t_bill, real_10Y, dy), start),end_d)
assets_names = colnames(data)
T, n_assets = size(data)
model = VARModel(data)
model_summary(model)

,const,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,Dividend_Yeld,std
USA_Net_T_Bill,0.1683,0.125,0.0203,-0.0207,2.4105,-4.0492,0.035,0.0724
World_ex_USA_T_Bill,-0.0243,0.0265,0.0179,0.0576,0.2398,-1.2235,-0.0083,0.0845
EM_Net_T_Bill,0.2286,-0.2143,0.2693,0.0241,1.3845,-1.6515,0.0528,0.1275
T_Bill_CPI,0.0276,-0.0004,0.0095,-0.0027,0.9043,-0.6602,0.0064,0.0081
Yeld_10Y_CPI,0.0296,-0.0021,0.0084,-0.0014,-0.0064,0.2439,0.0068,0.008
Dividend_Yeld,-0.2241,0.0246,-0.1051,0.0344,-3.7336,2.7902,0.9473,0.0736


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,Dividend_Yeld
USA_Net_T_Bill,1.0,0.8,0.65,-0.14,-0.11,-0.59
World_ex_USA_T_Bill,0.8,1.0,0.74,-0.26,-0.24,-0.48
EM_Net_T_Bill,0.65,0.74,1.0,-0.26,-0.22,-0.32
T_Bill_CPI,-0.14,-0.26,-0.26,1.0,0.99,0.08
Yeld_10Y_CPI,-0.11,-0.24,-0.22,0.99,1.0,0.07
Dividend_Yeld,-0.59,-0.48,-0.32,0.08,0.07,1.0


In [61]:
scenarios = simulate(model, 100, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(periods), string.(assets_names[a]))
end  

,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,Dividend_Yeld
1,0.0732,0.0278,0.0191,0.0225,0.0121,-16.9971
5,0.0802,0.0414,0.0068,0.007,0.0046,-17.1192
15,0.0713,0.0401,0.0094,0.0011,0.0051,-16.9424
25,0.0684,0.0369,0.0131,0.0,0.0063,-16.7905


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,Dividend_Yeld
1,0.1357,0.159,0.2303,0.017,0.0163,0.2658
5,0.0697,0.0865,0.1169,0.0115,0.0096,0.5668
15,0.0403,0.0541,0.0715,0.0078,0.0076,0.6924
25,0.0314,0.0434,0.0574,0.0061,0.0068,0.6735


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,Dividend_Yeld
1,0.02,-0.009,0.027,-0.0196,-0.0094,0.0073
5,0.0086,-0.0081,0.0058,-0.0286,-0.0057,-0.03
15,0.022,0.0115,-0.0129,0.0375,-0.0173,-0.0156
25,0.0575,0.0346,0.0172,0.0136,-0.0407,-0.0155


,USA_Net_T_Bill,World_ex_USA_T_Bill,EM_Net_T_Bill,T_Bill_CPI,Yeld_10Y_CPI,Dividend_Yeld
1,0.0083,0.0185,-0.0457,0.0527,0.0904,-0.0325
5,-0.034,0.0002,0.0542,0.0384,0.0241,0.0414
15,-0.0948,-0.051,0.048,0.0785,-0.1041,-0.0572
25,0.0246,0.0533,0.0878,0.0103,-0.0363,-0.0457


,0.03,0.25,0.5,0.75,0.97
1,-0.182,-0.0173,0.0726,0.1641,0.3294
5,-0.0492,0.0326,0.08,0.1281,0.2095
15,-0.0039,0.0438,0.0709,0.0986,0.1472
25,0.011,0.047,0.0677,0.0892,0.1287


,0.03,0.25,0.5,0.75,0.97
1,-0.2652,-0.0792,0.0281,0.1333,0.3247
5,-0.1204,-0.017,0.0415,0.1002,0.2042
15,-0.0602,0.0031,0.0399,0.0767,0.1419
25,-0.0436,0.0075,0.0366,0.0662,0.1183


,0.03,0.25,0.5,0.75,0.97
1,-0.4097,-0.1392,0.0183,0.1726,0.4578
5,-0.2141,-0.0718,0.007,0.0838,0.2272
15,-0.1237,-0.0398,0.0097,0.0581,0.1438
25,-0.0936,-0.0248,0.0131,0.0512,0.1226


,0.03,0.25,0.5,0.75,0.97
1,-0.0098,0.0111,0.0226,0.0338,0.0539
5,-0.0146,-0.0005,0.0071,0.0148,0.0287
15,-0.0134,-0.004,0.0011,0.0062,0.0159
25,-0.0115,-0.0041,0.0,0.0041,0.0118


,0.03,0.25,0.5,0.75,0.97
1,-0.0183,0.0014,0.0121,0.023,0.0428
5,-0.0134,-0.0019,0.0046,0.011,0.0228
15,-0.0093,0.0,0.005,0.0102,0.0192
25,-0.0067,0.0017,0.0063,0.011,0.0189


,0.03,0.25,0.5,0.75,0.97
1,-17.5047,-17.1783,-16.997,-16.8179,-16.4948
5,-18.1914,-17.4992,-17.1137,-16.7384,-16.0631
15,-18.2551,-17.41,-16.9356,-16.4653,-15.6302
25,-18.0528,-17.2487,-16.7793,-16.3343,-15.5182


In [75]:
start = Date(1989, 01, 01)
end_d = Date(2025, 01, 31)
assets_names = [:USA_Net_CPI, :World_ex_USA_CPI, :EM_Net_CPI]
data = to(from(merge(real_returns[assets_names],  real_t_bill, real_10Y, cpi), start),end_d)
assets_names = colnames(data)
T, n_assets = size(data)
model = VARModel(data)
model_summary(model)

,const,USA_Net_CPI,World_ex_USA_CPI,EM_Net_CPI,T_Bill_CPI,Yeld_10Y_CPI,CPI,std
USA_Net_CPI,0.0229,0.1178,-0.0046,-0.0073,2.4583,-3.0786,0.6927,0.0722
World_ex_USA_CPI,0.0236,0.0214,0.026,0.0667,1.5608,-3.1879,-1.0979,0.0821
EM_Net_CPI,0.014,-0.2102,0.2202,0.0472,0.8476,-0.2693,0.2972,0.1256
T_Bill_CPI,-0.0068,0.0036,0.0063,-0.0093,0.6303,0.2691,0.8673,0.0074
Yeld_10Y_CPI,-0.0063,0.0021,0.0048,-0.0076,-0.2815,1.1642,0.8476,0.0073
CPI,0.0069,-0.005,-0.0041,0.0089,0.3304,-0.2685,0.1113,0.0076


,USA_Net_CPI,World_ex_USA_CPI,EM_Net_CPI,T_Bill_CPI,Yeld_10Y_CPI,CPI
USA_Net_CPI,1.0,0.79,0.64,-0.03,0.01,0.05
World_ex_USA_CPI,0.79,1.0,0.72,-0.16,-0.14,0.19
EM_Net_CPI,0.64,0.72,1.0,-0.2,-0.15,0.21
T_Bill_CPI,-0.03,-0.16,-0.2,1.0,0.99,-0.99
Yeld_10Y_CPI,0.01,-0.14,-0.15,0.99,1.0,-0.99
CPI,0.05,0.19,0.21,-0.99,-0.99,1.0


In [76]:
scenarios = simulate(model, 100, 10_000)
periods = [1,5, 15, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))

for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.03, .25, .5, .75, .97], string.(periods), string.(assets_names[a]))
end  

,USA_Net_CPI,World_ex_USA_CPI,EM_Net_CPI,T_Bill_CPI,Yeld_10Y_CPI,CPI
1,0.1158,0.0669,0.0777,0.0229,0.0199,0.0264
5,0.1042,0.0402,0.0643,0.013,0.0172,0.0293
15,0.0872,0.0317,0.0575,0.0051,0.0152,0.0279
25,0.0822,0.031,0.0553,0.0016,0.0136,0.0272


,USA_Net_CPI,World_ex_USA_CPI,EM_Net_CPI,T_Bill_CPI,Yeld_10Y_CPI,CPI
1,0.1323,0.1521,0.227,0.0131,0.0129,0.0137
5,0.0713,0.0784,0.1183,0.0083,0.0088,0.0075
15,0.044,0.0455,0.0698,0.0081,0.0072,0.0048
25,0.0348,0.0351,0.0539,0.0074,0.0062,0.0039


,USA_Net_CPI,World_ex_USA_CPI,EM_Net_CPI,T_Bill_CPI,Yeld_10Y_CPI,CPI
1,-0.0198,-0.0072,-0.0292,-0.0278,-0.0202,0.0088
5,-0.0194,0.0023,-0.0299,0.0184,0.0127,-0.0124
15,0.0126,0.0523,0.0058,0.0044,0.0102,0.0107
25,0.0169,0.0207,-0.0217,-0.0148,-0.0085,0.0328


,USA_Net_CPI,World_ex_USA_CPI,EM_Net_CPI,T_Bill_CPI,Yeld_10Y_CPI,CPI
1,0.018,0.0197,0.0779,-0.0168,-0.0105,0.0176
5,0.054,0.1062,0.0157,-0.003,0.0217,-0.0304
15,-0.0586,0.0077,0.0092,-0.0796,-0.0089,-0.0397
25,-0.0169,0.054,0.0227,-0.0025,-0.0102,0.0348


,0.03,0.25,0.5,0.75,0.97
1,-0.1339,0.0262,0.1161,0.205,0.3601
5,-0.0318,0.0556,0.104,0.1529,0.2369
15,0.0036,0.0573,0.087,0.117,0.1698
25,0.0165,0.0589,0.0823,0.1054,0.1474


,0.03,0.25,0.5,0.75,0.97
1,-0.2239,-0.0365,0.0683,0.169,0.3501
5,-0.1076,-0.0111,0.0401,0.0923,0.1877
15,-0.053,0.0005,0.0314,0.0618,0.1179
25,-0.0348,0.0078,0.0306,0.0543,0.0971


,0.03,0.25,0.5,0.75,0.97
1,-0.3556,-0.0724,0.0796,0.2274,0.5042
5,-0.1593,-0.0149,0.066,0.1444,0.2858
15,-0.0754,0.0105,0.058,0.1039,0.1894
25,-0.0464,0.0184,0.0558,0.0916,0.155


,0.03,0.25,0.5,0.75,0.97
1,-0.0021,0.0143,0.0229,0.0316,0.0475
5,-0.0024,0.0075,0.0129,0.0186,0.0287
15,-0.01,-0.0004,0.005,0.0108,0.0202
25,-0.0121,-0.0034,0.0016,0.0066,0.0154


,0.03,0.25,0.5,0.75,0.97
1,-0.0048,0.0112,0.02,0.0287,0.044
5,0.0007,0.0113,0.0171,0.023,0.0339
15,0.0018,0.0103,0.0152,0.0201,0.0288
25,0.002,0.0094,0.0136,0.0179,0.0251


,0.03,0.25,0.5,0.75,0.97
1,0.0005,0.0173,0.0263,0.0355,0.0523
5,0.0152,0.0243,0.0294,0.0344,0.0434
15,0.0187,0.0246,0.0278,0.0312,0.0372
25,0.0199,0.0246,0.0272,0.0299,0.0345
